# Generate emoji_list

In [1]:
import re
import numpy as np
import pandas as pd

# Load data

In [2]:
# Has annotations. Hard to import from URL. No skin tone.
#EMOJI_URL = 'https://www.unicode.org/emoji/charts/emoji-list.html'
# Full list, only name. No skin tone modifiers.
EMOJI_URL = 'https://www.unicode.org/emoji/charts/full-emoji-list.html'
# The skin tone modifiers.
SKIN_URL = 'https://www.unicode.org/emoji/charts/full-emoji-modifiers.html'


In [58]:
# Might have to play with the number of headers. A list of dataframes will be returned.
COLUMNS = [u'№', 'Code', 'Browser', 'CLDR Short Name']

emoji_list = pd.read_html(EMOJI_URL, encoding='utf8', header=2)

# Load in skin and other modifier data
emoji_list_mod = pd.read_html(SKIN_URL, encoding='utf8', header=2)


In [85]:
# Merge dataframe
df = emoji_list[0].append(emoji_list_mod[0])
df = df.rename(columns={u'№': 'index'})

In [86]:
# If google code that applies to multiple columns then overwrite the name. "... ..."
def fill_name_col(row):
    ''' If the CLDR Short Name is blank (the last column), then overwrite with google column.'''
    if pd.isna(row[-1]):
        row[-1] = row[4]
    return row 
        
df = df.apply(fill_name_col, axis=1) 

In [96]:
def only_keep_int(text):
    ''' Return int contained in a string or NaN'''
    ints_found = re.findall('\d+', text)
    if not isinstance(text, str):
        return np.nan
    
    if ints_found:
        return int(ints_found[0])
    else:
        return np.nan

# Convert index to int or nan
df['index'] = df['index'].apply(only_keep_int)
# Drop nan in the index.
df = df.dropna(subset=['index']).dropna(axis=1)


In [99]:
df = df.rename(columns={'Browser': 'Unicode', 
                        'CLDR Short Name': 'Name'})
# Ignore annotations for now. Can ge
df['Annotations'] = ''
df = df.reset_index(drop=True)
df.head()

,index,Code,Unicode,Name,Annotations
0,1.0,U+1F600,😀,grinning face,
1,2.0,U+1F603,😃,grinning face with big eyes,
2,3.0,U+1F604,😄,grinning face with smiling eyes,
3,4.0,U+1F601,😁,beaming face with smiling eyes,
4,5.0,U+1F606,😆,grinning squinting face,


# Write file

In [100]:
df.to_csv('emoji_list.csv', encoding='utf8')